# Get Chemical Identifiers: LCIA QSAR Project
**Author:** Jacob Kvasnicka <br>
**Date:** June 24, 2023

Get chemical identifiers to use as inputs to the OPERA 2.9 user interface.

In [1]:
import pandas as pd
from os import path 
import re

import pattern
from config_management import UnifiedConfiguration
from raw_processing import RawDataProcessor

In [2]:
config_mapping_path = 'Input\configuration-mapping.json'
config = UnifiedConfiguration(config_mapping_path)
raw_processor = RawDataProcessor(config.path)

In [3]:
identifiers = raw_processor.get_labeled_identifiers()

identifiers

0       DTXSID5020281
1       DTXSID8020961
2       DTXSID0021834
3       DTXSID3032622
4       DTXSID2044347
            ...      
8313    DTXSID5057882
8314    DTXSID5057884
8315    DTXSID0057885
8316    DTXSID1057905
8317    DTXSID6057906
Name: dtxsid, Length: 6598, dtype: object

In [4]:
all_identifiers = raw_processor.get_all_identifiers()

all_identifiers

0          DTXSID9047623
1          DTXSID0052700
2         DTXSID00583560
3         DTXSID00859050
4         DTXSID00860464
               ...      
479861     DTXSID3038307
479862     DTXSID8038300
479863     DTXSID8074158
479864     DTXSID2032180
479865     DTXSID7020895
Name: DTXSID, Length: 479866, dtype: object